In [1]:
import os
import random
import math
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from einops.layers.torch import Rearrange
from einops import repeat

In [2]:
# デバイスの設定
if torch.cuda.is_available():
    DEVICE = "cuda"
elif torch.mps.is_available():
    DEVICE = "mps"
else:
    DEVICE = "cpu"
print(f"Using device: {DEVICE}")

Using device: mps


# GPTモデルの実装

In [3]:
class SelfAttention(nn.Module):
    """ セルフアテンションを実装 """
    def __init__(self, config, resid_pdrop=0.1, attn_pdrop=0.1, causal=True) -> None:
        """
        Args:
            config: Config
            resid_pdrop : float
                出力projection層のドロップアウト率
            attn_pdrop : float
                Attentionのドロップアウト率
            causal : bool
                causal maskを利用するかどうか判別するフラグ
        """
        super().__init__()
        assert config.n_embd % config.n_head == 0

        # 入力をK, Q, Vにそれぞれ変換する全結合層
        self.key = nn.Linear(config.n_embd, config.n_embd)
        self.query = nn.Linear(config.n_embd, config.n_embd)
        self.value = nn.Linear(config.n_embd, config.n_embd)

        self.attn_drop = nn.Dropout(attn_pdrop)
        self.resid_drop = nn.Dropout(resid_pdrop)

        # Multi-Head Attentionアウトプットの全結合層
        self.proj = nn.Linear(config.n_embd, config.n_embd)

        # torch.trilは行列の右上三角部分をゼロにして返す（予測するトークンの右側をマスク）
        # nn.Moduleのregister_bufferは, モデルのパラメータとならないtensorを追加するのに使われる
        if causal:
            self.register_buffer(
                name="mask",
                tensor=torch.tril(
                    torch.ones(config.block_size, config.block_size)
                ).view(1, 1, config.block_size, config.block_size)
            )
        self.n_head = config.n_head

    def forward(self, x, layer_past=None):
        """
        Args:
            x : torch.Tensor ( b, t, d )
                入力ベクトル系列
                b : バッチサイズ
                t : シークエンス長. コンテクストサイズ (block_size)よりも小さくないといけない
                d : Embedding次元数. 上図のd_model
        Returns:
            y : torch.Tensor ( b, t, d )
        """
        b, t, d = x.size()

        # Key, Que, Valueをそれぞれの全結合層で計算
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        # Multi-Head．d_kやd_vがd_model // n_headsになるような実装だが，必ずしもその必要はない
        k = k.view(b, t, self.n_head, d // self.n_head).transpose(1, 2)
        q = q.view(b, t, self.n_head, d // self.n_head).transpose(1, 2)
        v = v.view(b, t, self.n_head, d // self.n_head).transpose(1, 2)

        # QとKの行列積をとり, sqrt(d_k)でスケール
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))

        # Causal mask
        if hasattr(self, "mask"):
            att = att.masked_fill(self.mask[:, :, :t, :t] == 0, float('-inf'))

        att = F.softmax(att, dim=-1)

        att = self.attn_drop(att)

        # Attention mapとValuesの行列積
        y = att @ v

        # 各headからの出力を結合
        y = y.transpose(1, 2).contiguous().view(b, t, d)

        # Attention出力のprojection層
        y = self.resid_drop(self.proj(y)) # ( b, t, embd_dim )

        return y

## Blockの定義

In [ ]:
class Block(nn.Module):
    def __init__(self, config, resid_pdrop=0.1, causal=True) -> None:
        """
        Args:
            config (Config): 設定
            resid_pdrop (float, optional): ドロップアウト確率. Defaults to 0.1.
            causal (bool, optional): 読み取り方向. Defaults to True.
        """
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.attn = SelfAttention(config, causal=causal)

        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(resid_pdrop),
        )

    def forward(self, x):
        """
        Args:
            x: torch.Tensor (b, t, d)
        """
        # LinearNormalizationはAttentionやFeedForwardの前に適用する
        # プラス、残差接続
        x = self.attn(self.ln_1(x)) + x
        x = self.mlp(self.ln_2(x)) + x
        return x

# GPTの定義

In [5]:
class GPT(nn.Module):
    def __init__(self, config, embd_pdrop=0.1) -> None:
        """
        Args:
            config (Config): 設定
            embd_pdrop (float, optional): 埋め込みのドロップアウト確率. Defaults to 0.1.
        """
        super().__init__()
        self.config = config

        # 文字の表現と位置の表現をつなぐルックアップテーブル
        self.tok_embd = nn.Embedding(config.vocab_size, config.n_embd)
        # Positional encodingで足すベクトルはゼロで初期化し、学習可能なnn.Parameterとして登録する
        self.pos_embd = nn.Parameter(torch.zeros(1, config.block_size, config.n_embd))
        # 埋め込みのドロップアウト
        self.drop_embd = nn.Dropout(embd_pdrop)
        # n_layer個のブロックをnn.Sequentialで連結する
        self.blocks = nn.Sequential(*[Block(config) for _ in range(config.n_layer)])
        # デコーダのhead
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.block_size = config.block_size

        # 引数で与えた関数をモデルのモジュールについて再起的に適用
        self.apply(self._init_weights)

    def forward(self, idx, targets=None):
        """
        Args:
            idx: torch.Tensor (b, t)
                入力のトークンID
            targets: torch.Tensor (b, t)
                目標のトークンID
        """
        b, t = idx.shape
        # 入力シーケンスがコンテキストサイズを超えていないかチェック
        assert t <= self.config.block_size, "フォワードできません"

        # 文字のidxを表現ベクトルに変換
        token_embd = self.tok_embd(idx)  # (b, t, d)
        position_embd = self.pos_embd[:, :t, :]  # (1, t, d)
        x = self.drop_embd(token_embd + position_embd)

        # transformerのブロックを順番に適用
        x = self.blocks(x)  # (b, t, d)

        # GPT-2で追加された最後のlayer norm
        x = self.ln_f(x)  # (b, t, d)

        logits = self.head(x)  # (b, t, vocab_size)

        # 訓練の時はロスを計算
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            loss = None

        return logits, loss

    def get_block_size(self):
        return self.block_size

    def _init_weights(self, module):
        """ nn.Moduleのクラスメソッドapplyから呼び，moduleに関して再起的に適用 """
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def configure_optimizers(self):
        """
        各種パラメータを, AdamWによる更新式にL2正則化項を加えるものと加えないもの
        (biases, layer norm / embedding weights)の2グループに分け, 最後に
        PyTorchのoptimizerを返している. あまり重要ではない
        """
        decay = set()  # L2正則化をかけるパラメータ
        no_decay = set()  # L2正則化をかけないパラメータ
        whitelist_weight_modules = (nn.Linear,)  # L2正則化をかけるべきパラメータの型
        black_list_weight_modules = (nn.LayerNorm, nn.Embedding)

        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn # full param name

                if pn.endswith('bias'):
                    # biasは正則化しない
                    no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    # L2正則化をかける
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, black_list_weight_modules):
                    # L2正則化をかけない
                    no_decay.add(fpn)
                else:
                    # それ以外は正則化をかける
                    decay.add(fpn)

        # position embeddingのパラメータは正則化しない
        no_decay.add('pos_embd')

        # 見過ごされたパラメータがないかチェック
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        assert len(inter_params) == 0, "見過ごされたパラメータがあります"
        assert len(param_dict.keys() - union_params) == 0, "見落とされたパラメータがあります"

        optim_groups = [
            {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": train_config.weight_decay},
            {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0},
        ]
        optimizer = torch.optim.AdamW(optim_groups, lr=train_config.learning_rate, betas=train_config.betas)
        return optimizer
